In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from src.augmentations import CustomAugmentations

import numpy as np
import random
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)


In [2]:
height,width=32,32
batch_size=64

In [3]:
from src.augmentations.CustomAugmentations import CustomAugmentationsTF

# Define the augmentation probabilities
p_dict = {
  #  "flip": 0.5,
    "transpose": 0.5,
    "gauss_noise": 0.2,
    "brightness_contrast": 0.2,
    "hue_saturation_value": 0.2,
}

p_dicts = []
for aug_type in p_dict.keys():
    p_d = {aug_type: .5}
    p_dicts.append(p_d)
p_dicts.append(p_dict)

p_dicts


[{'transpose': 0.5},
 {'gauss_noise': 0.5},
 {'brightness_contrast': 0.5},
 {'hue_saturation_value': 0.5},
 {'transpose': 0.5,
  'gauss_noise': 0.2,
  'brightness_contrast': 0.2,
  'hue_saturation_value': 0.2}]

In [4]:
import matplotlib.pyplot as plt

augmented = original_dataset.map(lambda image, label: (custom_aug.augment(image), label))



In [5]:
from copy import deepcopy
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm.notebook import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

from src.kerasCNN import *
import pickle


EPOCHS = 50

histories = list()
accuracies = list()
class_accuracies = list()
conf_mats = list()

for augs in tqdm(p_dicts, desc="Augmentation"):
    aug_str = "_".join([key + str(value) for key, value in augs.items()])
    print(f"Augmentations: {aug_str}")

    for iteration in tqdm(range(5), desc="Iteration"):
        train_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/train",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="training",
        )

        val_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/valid",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="validation",
        )

        test_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/test",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=False,
        )

        dnn_model = create_cnn(
            kernel_sizes=[3, 3, 3,],
            num_filters=[32, 64, 64],
            fc_sizes=[128, 64, 64],
        )
       

        history = {
            "loss": list(),
            "accuracy": list(),
            "val_loss": list(),
            "val_accuracy": list(),
        }

        print(f"Fitting model {iteration}")
        for epoch in tqdm(range(EPOCHS), desc="Epoch"):
            print(f"Epoch {epoch + 1}/{EPOCHS}")
            custom_aug = CustomAugmentationsTF(augs)
            train_set_aug = train_set.map(lambda x, y: (custom_aug.augment(x), y))

            hist_ = dnn_model.fit(train_set_aug, validation_data=val_set, epochs=1)
            history["loss"].append(hist_.history["loss"])
            history["accuracy"].append(hist_.history["accuracy"])
            history["val_loss"].append(hist_.history["val_loss"])
            history["val_accuracy"].append(hist_.history["val_accuracy"])

        print(f"Predicting model {iteration}")
        preds = dnn_model.predict(test_set, verbose=2)
        preds = preds.argmax(axis=1)
        classes = test_set.class_names
        test_labels = list()
        for images, labels in test_set:
            class_labels = [int(label) for label in labels]
            test_labels.extend(class_labels)
        test_labels = np.array(test_labels)

        conf_mat = confusion_matrix(test_labels, preds)
        accuracy = accuracy_score(test_labels, preds)
        print(f"Accuracy: {accuracy} for model {iteration}")
        class_accuracy = conf_mat.diagonal() / conf_mat.sum(axis=1)

        print(f"Saving model {iteration}")
        dnn_model.save(f"../models/BESTClassicCNN_aug_{aug_str}_{iteration}.h5")
        histories.append(deepcopy(history))
        accuracies.append(deepcopy(accuracy))
        class_accuracies.append(deepcopy(class_accuracy))
        conf_mats.append(deepcopy(conf_mat))


    with open(f"results/HISTORY_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(histories, f)

    with open(f"results/ACCURACY_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(accuracies, f)

    with open(f"results/CLASS_ACCURACY_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(class_accuracies, f)

    with open(f"results/CONF_MAT_BESTClassicCNN_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(conf_mats, f)

Augmentation:   0%|          | 0/5 [00:00<?, ?it/s]

Augmentations: transpose0.5


Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Found 90000 files belonging to 10 classes.
Using 85500 files for training.
Found 90000 files belonging to 10 classes.
Using 4500 files for validation.
Found 90000 files belonging to 10 classes.
Fitting model 0


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/50
1336/1336 [==============================] - 19s 12ms/step - loss: 1.6243 - accuracy: 0.4014 - val_loss: 1.5687 - val_accuracy: 0.4196
Epoch 2/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.4697 - accuracy: 0.4623 - val_loss: 2.2368 - val_accuracy: 0.2476
Epoch 3/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.3495 - accuracy: 0.5114 - val_loss: 1.6474 - val_accuracy: 0.4333
Epoch 4/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.2126 - accuracy: 0.5655 - val_loss: 1.3205 - val_accuracy: 0.5253
Epoch 5/50
1336/1336 [==============================] - 16s 12ms/step - loss: 1.3046 - accuracy: 0.5275 - val_loss: 1.9100 - val_accuracy: 0.3469
Epoch 6/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.1696 - accuracy: 0.5805 - val_loss: 2.4115 - val_accuracy: 0.2620
Epoch 7/50
1336/1336 [==============================] - 16s 12ms/step - loss: 1.1012 - accuracy: 0.6064 - val_loss: 2.3319 -

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/50
1336/1336 [==============================] - 16s 11ms/step - loss: 1.6288 - accuracy: 0.4015 - val_loss: 2.1897 - val_accuracy: 0.2313
Epoch 2/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.3517 - accuracy: 0.5120 - val_loss: 2.3811 - val_accuracy: 0.2164
Epoch 3/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.4158 - accuracy: 0.4870 - val_loss: 1.3486 - val_accuracy: 0.5058
Epoch 4/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.2337 - accuracy: 0.5582 - val_loss: 1.2863 - val_accuracy: 0.5433
Epoch 5/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.2784 - accuracy: 0.5383 - val_loss: 2.0785 - val_accuracy: 0.3062
Epoch 6/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.1989 - accuracy: 0.5706 - val_loss: 1.2213 - val_accuracy: 0.5636
Epoch 7/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.0926 - accuracy: 0.6120 - val_loss: 1.2115 -

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/50
1336/1336 [==============================] - 17s 12ms/step - loss: 1.6215 - accuracy: 0.4022 - val_loss: 2.2898 - val_accuracy: 0.2078
Epoch 2/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.4480 - accuracy: 0.4719 - val_loss: 1.6311 - val_accuracy: 0.4440
Epoch 3/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.2623 - accuracy: 0.5475 - val_loss: 1.3696 - val_accuracy: 0.5067
Epoch 4/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.3326 - accuracy: 0.5179 - val_loss: 2.4340 - val_accuracy: 0.2724
Epoch 5/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.1856 - accuracy: 0.5756 - val_loss: 2.4438 - val_accuracy: 0.2542
Epoch 6/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.2363 - accuracy: 0.5566 - val_loss: 1.1937 - val_accuracy: 0.5662
Epoch 7/50
1336/1336 [==============================] - 15s 11ms/step - loss: 1.1673 - accuracy: 0.5829 - val_loss: 2.1307 -